시중 판매 중인 TV정보를 온라인 쇼핑몰 웹페이지에서 크롤링 하여 분석을 하고자 한다. 해당 데이터를 기반으로 각 TV의 제원은 어떠 한지 제원에 따른 평가는 어떠 한지 알아보고 향후 신제품 출시에 참고하고자 한다. (각 문제에서 전처리한 데이터는 다음 문제에 영향을 끼치지 않는다.)

In [14]:
import pandas as pd
df_origin = pd.read_csv("../Datasets/TV.csv")
df_origin.head(2)

,Product_Name,Stars,Ratings,Reviews,current_price,MRP,channel,Operating_system,Picture_qualtiy,Speaker,Frequency
0,Croma,4.2,1773,217,7990,20000,HD Ready 1366 x 768 Pixels,20 Speaker Output,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty
1,Adsun,3.8,6742,930,8699,21999,Netflix|Disney+Hotstar|Youtube,Operating System: Android Based,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate


In [15]:
df_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666 entries, 0 to 665
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_Name      666 non-null    object 
 1   Stars             666 non-null    float64
 2   Ratings           666 non-null    int64  
 3   Reviews           666 non-null    int64  
 4   current_price     666 non-null    int64  
 5   MRP               666 non-null    int64  
 6   channel           666 non-null    object 
 7   Operating_system  666 non-null    object 
 8   Picture_qualtiy   666 non-null    object 
 9   Speaker           666 non-null    object 
 10  Frequency         666 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 57.4+ KB


In [16]:
df_origin=df_origin.rename(columns={'Picture_qualtiy':'Picture_quality'})

#### Q1.주사율(Frequency)은 TV 제원중 가격에 큰 영향을 미치는 제원이다. 그런데 데이터 수집 중 실수가 발생하여 주사율(Frequency) 값 중 일부가 해상도(Picture_quality), 스피커(Speaker) 변수에 잘못 입력된 것이 발견되었다. 세 변수에 흩어져 있는 주사율 데이터를 취합하여 전체 TV 중에서 주사율이 60Hz인 TV는 총 몇 대 인가?  
※ 주사율은 2~3자리 숫자 뒤에 반드시 Hz 라는 단위로 표기되어 있다.  
※ 세 변수 모두 주사율 정보가 없을 경우 결측치로 간주하고 이를 분석에서 제외하시오.  
(정답 예시: 12)

In [17]:
df1 = df_origin.copy()

> deep copy `df.copy(deep=True)`
라고 하며 원본과는 완전하게 별개인 복사본이 생성됩니다. 사본과 원본의 수정은 서로에게 영향을 끼치지 않습니다.

In [18]:
df1.head(2)

,Product_Name,Stars,Ratings,Reviews,current_price,MRP,channel,Operating_system,Picture_quality,Speaker,Frequency
0,Croma,4.2,1773,217,7990,20000,HD Ready 1366 x 768 Pixels,20 Speaker Output,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty
1,Adsun,3.8,6742,930,8699,21999,Netflix|Disney+Hotstar|Youtube,Operating System: Android Based,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate


In [19]:
df1['Picture_quality']

0                    60 Hz Refresh Rate
1            HD Ready 1366 x 768 Pixels
2            HD Ready 1366 x 768 Pixels
3            HD Ready 1366 x 768 Pixels
4            HD Ready 1366 x 768 Pixels
                     ...               
661                  60 Hz Refresh Rate
662    Ultra HD (4K) 3840 x 2160 Pixels
663                  60 Hz Refresh Rate
664                   16 Speaker Output
665                  50 Hz Refresh Rate
Name: Picture_quality, Length: 666, dtype: object

> 아 0, 661보니깐 저런식으로 잘못 수집되어 있구나

In [20]:
df1 = df1[["Picture_quality", "Speaker", "Frequency"]].copy()
df1.head(2)

,Picture_quality,Speaker,Frequency
0,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty
1,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate


- `Hz` 없는 애들 삭제,(세 변수 모두 주사율 정보가 없을 경우 결측치로 간주하고 이를 분석에서 제외하시오.)

Q. map을 이용해서 'Picture_quality'에서 `Hz` 를 포함하는 행의 개수는?

In [21]:
df1['Picture_quality'].map(lambda x: 'Hz' in x).sum()

99

Q. 전체 컬럼에 적용하는 방법은?

In [9]:
df1.apply(lambda x: x.str.contains('Hz')).sum()

Picture_quality     99
Speaker             23
Frequency          540
dtype: int64

> lambda 함수가 컬럼별로 적용이 된다.

In [69]:
df1.shape

(666, 3)

In [70]:
df1.apply(lambda x: x.str.contains('Hz')).sum(axis=0)

Picture_quality     99
Speaker             23
Frequency          540
dtype: int64

In [71]:
df1.apply(lambda x: x.str.contains('Hz')).sum(axis=0).shape

(3,)

> 변하는 차원의 숫자를 넣으면 된다. 0은 행, 1은 컬럼 이런식으로 외우면 나중에 차원 많은 거 할 때 성립이 안됨, 2차원만 다룰 거면 그렇게 해도 됨

Q. 전체 컬럼을 다 더 할 때는 axis 몇을 써야 할까?

In [78]:
df1.apply(lambda x: x.str.contains('Hz')).sum(axis=1) #axis넣기전에 떠올려보자

0      1
1      1
2      1
3      1
4      1
      ..
661    1
662    1
663    1
664    1
665    1
Length: 666, dtype: int64

> 컬럼을 다 더하면 결국 행만 남을 거고 변하는 차원은 shape에서 1번째가 되겠죠 그래서 axis 1을 넣었다ㅡ



In [79]:
df1.apply(lambda x: x.str.contains('Hz')).sum(axis=1).shape

(666,)

In [72]:
df1[df1.apply(lambda x: x.str.contains('Hz')).sum(axis=1)==0]

,Picture_quality,Speaker,Frequency
234,Ultra HD (4K) 3840 x 2160 Pixels,20 W Speaker Output,3 x HDMI | 2 x USB
237,Ultra HD (4K) 3840 x 2160 Pixels,20 W Speaker Output,3 x HDMI | 2 x USB
335,40 W Speaker Output,4 x HDMI | 3 x USB,1 Year Manufaturer Warranty
547,Ultra HD (4K) 3840 x 2160 Pixels,20 W Speaker Output,3 x HDMI | 2 x USB


> 컬럼별로 Hz가 없는 애들은 삭제한 것이다

In [80]:
df1_dropedNA=df1.loc[~(df1.apply(lambda x: x.str.contains('Hz')).sum(axis=1)==0),]

In [84]:
df1_dropedNA.apply(lambda x: x.str.contains('60 Hz')).sum(axis=1).value_counts()

1    510
0    152
dtype: int64

### A1. 510

---

#### Q2. TV의 해상도(Picture_quality) 는 HD, 4K, 8K 로 나뉜다. 최근 시장에 등장한 8K 제품군은 상대적으로 비싸 비교적 매출이 떨어지는 편이다. 하지만 8K 제품군 구매자가 4K 제품군 구매자 대비 얼마나 만족하고 있는지 확인해보고자 한다. 8K 제품군의 평가 평균 점수(Stars)의 평균값과 4K 제품군의 평가 평균 점수와의 평균값 차이의 절대값을 산출하시오.
※ 화면 해상도는 관련 변수에서 "HD", "4K", "8K"로 찾아낼 수 있다.  
※ 화면 해상도 정보는 "Operating_system", "channel", "Picture_quality" 변수에 흩어져 있다.  
※ 결과는 반올림하여 소수점 둘째 자리까지 출력하시오.  
(정답 예시: 0.12)

In [25]:
col_cate = ["Operating_system", "channel", "Picture_quality"]

In [26]:
df2=df_origin[col_cate].copy()
df2

,Operating_system,channel,Picture_quality
0,20 Speaker Output,HD Ready 1366 x 768 Pixels,60 Hz Refresh Rate
1,Operating System: Android Based,Netflix|Disney+Hotstar|Youtube,HD Ready 1366 x 768 Pixels
2,Operating System: WebOS,Netflix|Prime Video|Disney+Hotstar|Youtube,HD Ready 1366 x 768 Pixels
3,Operating System: Android,Netflix|Prime Video|Disney+Hotstar|Youtube,HD Ready 1366 x 768 Pixels
4,Operating System: Android,Netflix|Prime Video|Disney+Hotstar|Youtube,HD Ready 1366 x 768 Pixels
...,...,...,...
661,NO Speaker Output,Ultra HD (4K) 3840 x 2160 pixels Pixels,60 Hz Refresh Rate
662,Operating System: Linux based,Netflix|Prime Video|Youtube,Ultra HD (4K) 3840 x 2160 Pixels
663,20 Speaker Output,HD Ready 1366 x 786 Pixels,60 Hz Refresh Rate
664,Full HD 1920 x 1080 Pixels,Netflix|Prime Video|Apple TV|Disney+Hotstar|Yo...,16 Speaker Output


> 661, 662 저렇게(4K) 데이터가 수집되어 있나보네

In [41]:
df2.apply(lambda x:x.str.contains('4K|8K'))

,Operating_system,channel,Picture_quality
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
661,False,True,False
662,False,False,True
663,False,False,False
664,False,False,False


Q. 혹시 한 행에 4K, 8K가 되어 있는 행이 있을까?

In [42]:
is_contains_words=df2.apply(lambda x:x.str.contains('4K|8K')).sum(axis=1)>1

In [43]:
is_contains_words.sum()

0

A. 없다. 둘은 완벽히 나눠지는구나. 둘을 골라내야 하는 작업은 안해도 되겠다!

In [45]:
df2['is_4K'] = df2[col_cate].apply(lambda x:x.str.contains('4K')).sum(axis=1)
df2['is_8K'] = df2[col_cate].apply(lambda x:x.str.contains('8K')).sum(axis=1)

In [46]:
df2.head()

,Operating_system,channel,Picture_quality,is_4K,is_8K
0,20 Speaker Output,HD Ready 1366 x 768 Pixels,60 Hz Refresh Rate,0,0
1,Operating System: Android Based,Netflix|Disney+Hotstar|Youtube,HD Ready 1366 x 768 Pixels,0,0
2,Operating System: WebOS,Netflix|Prime Video|Disney+Hotstar|Youtube,HD Ready 1366 x 768 Pixels,0,0
3,Operating System: Android,Netflix|Prime Video|Disney+Hotstar|Youtube,HD Ready 1366 x 768 Pixels,0,0
4,Operating System: Android,Netflix|Prime Video|Disney+Hotstar|Youtube,HD Ready 1366 x 768 Pixels,0,0


In [48]:
df2_w_stars=pd.concat([df2[['is_4K','is_8K']],df_origin['Stars']], axis=1)
df2_w_stars

,is_4K,is_8K,Stars
0,0,0,4.2
1,0,0,3.8
2,0,0,4.4
3,0,0,4.3
4,0,0,4.3
...,...,...,...
661,1,0,4.6
662,1,0,4.5
663,0,0,0.0
664,0,0,0.0


In [49]:
mean_4k=df2_w_stars.loc[df2_w_stars.is_4K==1,'Stars'].mean()

In [50]:
mean_8k = df2_w_stars.loc[df2_w_stars.is_8K==1,'Stars'].mean()

In [51]:
round(abs(mean_4k-mean_8k),2)

0.38

### A2. 0.38

---

#### Q3. 좋은 평가를 받는 제품의 조건을 알아보고자 한다. 이를 위해 기존 변수와 더불어 여러 파생변수를 생성 후 해당 변수를 독립변수로 하고 평가 평균 점수(Stars)를 종속변수로 하여 Random Forest 분석을 통해 좋은 평가를 받는 제품의 조건을 알아보고자 한다. 아래 절차를 수행하여 변수 중요도를 확인하고, 가장 큰 값의 변수명(한글)을 쓰시오. ex) 후기 작성 비율, 공장 출고가

<독립변수>
 - 후기 작성 비율: 후기 수(Reviews)/ 평가 수(Ratings)
 - 공장 출고가(MRP)
 - 할인율: 현재 판매가(current_price) / 공장 출고가(MRP)
 - Netflix 제공 여부: 제공(1), 미제공(0)
 - Prime Video 제공 여부: 제공(1), 미제공(0)
 - 고해상도 여부: 4K 또는 8K(1), 나머지 0

<종속변수>
 - 평가 평균 점수(Stars)

※ channel 변수에 해상도(‘Pixel’ 문자 참조) 또는 운영체제 정보(‘Oper’ 문자 참조)가 있는 TV는 분석에서 제외하시오.  
※ Netflix와 Prime Video 제공여부 변수는 channel 변수를 참고하시오.  
※ 독립변수를 생성하면서 결측치가 생성되는 경우 해당 행을 모두 제거하시오.  
※ 학습 대상이 되는 행 개수는 197이다  
※ 고해상도 여부 변수 생성은 해상도(Picture_quality) 변수만 참고하여 생성하시오.  
※ seed는 123으로 지정하시오.  

---

※ channel 변수에 해상도(‘Pixel’ 문자 참조) 또는 운영체제 정보(‘Oper’ 문자 참조)가 있는 TV는 분석에서 제외하시오.

In [30]:
df3=df_origin.copy()

In [31]:
df3.channel

0                             HD Ready 1366 x 768 Pixels
1                         Netflix|Disney+Hotstar|Youtube
2             Netflix|Prime Video|Disney+Hotstar|Youtube
3             Netflix|Prime Video|Disney+Hotstar|Youtube
4             Netflix|Prime Video|Disney+Hotstar|Youtube
                             ...                        
661              Ultra HD (4K) 3840 x 2160 pixels Pixels
662                          Netflix|Prime Video|Youtube
663                           HD Ready 1366 x 786 Pixels
664    Netflix|Prime Video|Apple TV|Disney+Hotstar|Yo...
665              Ultra HD (4K) 3840 x 2160 pixels Pixels
Name: channel, Length: 666, dtype: object

In [52]:
df3['channel'].str.contains('Pixel|Oper').sum()

118

In [53]:
df3=df3.loc[~df3['channel'].str.contains('Pixel|Oper')]
df3.head(2)

,Product_Name,Stars,Ratings,Reviews,current_price,MRP,channel,Operating_system,Picture_quality,Speaker,Frequency
1,Adsun,3.8,6742,930,8699,21999,Netflix|Disney+Hotstar|Youtube,Operating System: Android Based,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
2,LG,4.4,38870,3443,16499,21990,Netflix|Prime Video|Disney+Hotstar|Youtube,Operating System: WebOS,HD Ready 1366 x 768 Pixels,10 W Speaker Output,50 Hz Refresh Rate


---

※ Netflix와 Prime Video 제공여부 변수는 channel 변수를 참고하시오.  

Netflix 제공 여부: 제공(1), 미제공(0)  
Prime Video 제공 여부: 제공(1), 미제공(0)

In [54]:
df3['channel'].str.contains('Netflix')

1      True
2      True
3      True
4      True
5      True
       ... 
655    True
656    True
657    True
662    True
664    True
Name: channel, Length: 548, dtype: bool

In [55]:
df3['channel'].str.contains('Netflix').astype(int)

1      1
2      1
3      1
4      1
5      1
      ..
655    1
656    1
657    1
662    1
664    1
Name: channel, Length: 548, dtype: int32

In [58]:
df3=df3.assign(support_netflix=df3['channel'].str.contains('Netflix').astype(int),
                support_prime=df3['channel'].str.contains('Prime Video').astype(int))

> assign으로도 컬럼 추가 가능

후기 작성 비율: 후기 수(Reviews)/ 평가 수(Ratings)  
할인율: 현재 판매가(current_price) / 공장 출고가(MRP)  
고해상도 여부: 4K 또는 8K(1), 나머지 0

In [59]:
df3=df3.assign(review_ratio = df3.Reviews/df_q3.Ratings,
                          discount_rate = df3.current_price/df_q3.MRP,
                          is_high = df3.Picture_quality.str.contains('4K|8K').astype(int)
                          )
# df3.Picture_quality.map(lambda x: 1 if '4K' in x or '8K' in x else 0).sum()

In [60]:
col_feature = ['support_netflix','support_prime','review_ratio','discount_rate','is_high','MRP']

※ 독립변수를 생성하면서 결측치가 생성되는 경우 해당 행을 모두 제거하시오.

In [61]:
df3[col_feature].apply(lambda x: x.isna().sum())

support_netflix      0
support_prime        0
review_ratio       351
discount_rate        0
is_high              0
MRP                  0
dtype: int64

In [62]:
df3_notna=df3.loc[~df3.review_ratio.isna()]

※ 학습 대상이 되는 행 개수는 197이다

In [63]:
len(df3_notna)

197

In [64]:
from sklearn.ensemble import RandomForestRegressor

In [65]:
model_rfr = RandomForestRegressor(random_state=123)
model_rfr.fit(X=df3_notna[col_feature],
                 y= df3_notna['Stars'])

RandomForestRegressor(random_state=123)

In [66]:
model_rfr.feature_importances_

array([0.02748716, 0.02455435, 0.31167862, 0.45073501, 0.0131053 ,
       0.17243956])

In [67]:
model_rfr.feature_names_in_

array(['support_netflix', 'support_prime', 'review_ratio',
       'discount_rate', 'is_high', 'MRP'], dtype=object)

In [68]:
pd.Series(index=model_rfr.feature_names_in_, data=model_rfr.feature_importances_).idxmax()

'discount_rate'

### A3. 할인율